In [55]:
import pandas as pd
from scipy import stats
from statistics import mean

In [100]:
internals = pd.read_csv("internals.csv")
externals = pd.read_csv("externals.csv")

In [4]:
internals.head()

,tnt_uuid,assessment_uuid,ttsess_uuid,stu_uuid,ttclass_uuid,status,name,activate,assessment_result_uuid,result_numeric,...,session_start,session_end,status.1,title,school_year,tnt_uuid.2,ldom_uuid.1,learning_domain,learning_domain_code,learning_domain_enum
0,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e3e986-bc32-0a22-9085-0d6088b6d309,67045c9c-21be-879c-1774-9b6a2d22a73b,3ff4e152-fe23-90f1-2dc5-795379649da8,443eb7a2-e3f6-ca47-669c-e72638cea15a,ASSIGNED,Mathematics Semester Result G08MT4 K,2020-11-26 14:00:00+00:00,c512cee0-6765-6cd8-64d8-63953fcec12c,0.96,...,2020-06-15,2020-12-09,active,Semester 2,2020,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,NaN
1,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e35d50-9b2f-f8ae-6555-58f0949d9b66,67046440-439d-5c20-d62f-8ddc655052cf,3ff4e152-fe23-90f1-2dc5-795379649da8,443d72a3-8eaa-c49e-bdf5-58efb72b9e32,ASSIGNED,Mathematics Semester Result G07MT1 K,2019-06-13 14:00:00+00:00,c508f21c-bdb6-5f80-3b7a-bacfdef687c6,0.97,...,2019-01-29,2019-06-14,active,Semester 1,2019,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,NaN
2,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e44eae-1534-f120-29e1-5d0abff52050,6704c9c1-2c77-0533-e04a-e73fa620c87b,3ff4e152-fe23-90f1-2dc5-795379649da8,443f3130-1ca9-65f0-852c-ab39aeb5be6f,ASSIGNED,Mathematics Semester One Grade G08MT4 K,2020-06-11 14:00:00+00:00,c508f942-d586-1a52-6d89-c228eaf745df,0.92,...,2020-01-31,2020-06-12,active,Semester 1,2020,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,NaN
3,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e49d22-9205-e2b7-afdc-312fd9564810,670498db-d726-8504-5ac1-d28ef68b5bcb,3ff4e152-fe23-90f1-2dc5-795379649da8,443e4335-7376-0acd-2d95-20751949a458,ASSIGNED,Mathematics Semester result G07MT1 K,2019-12-01 14:00:00+00:00,c50b8f96-1903-9cfa-4505-f01ca31d6f43,0.95,...,2019-06-17,2019-12-11,active,Semester 2,2019,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,NaN
4,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e46ea5-e4c8-a3f1-a72d-324c504d8fb3,67044b53-5747-9015-5532-332cc438841a,3ff4e152-fe23-90f1-2dc5-795379649da8,443fa9f5-a241-7aea-31b4-332332ae5592,ASSIGNED,Mathematics A Semester result G09MA4 K,2021-11-24 14:00:00+00:00,c51f0189-2fa0-f43e-48c3-3d5e3c155b60,0.87,...,2021-07-13,2021-12-10,active,Semester 2,2021,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,NaN


In [14]:
print(internals['stu_uuid'].nunique())
print(externals['stu_uuid'].nunique())

298
289


**Dropping students which don't have NAPLAN Numeracy records**

In [21]:
filt = internals['stu_uuid'].isin(externals['stu_uuid'].unique())
internals_with_NAPLAN = internals[filt]
internals_with_NAPLAN['stu_uuid'].nunique()

289

In [ ]:
internals_with_NAPLAN

**Types of assessments**

In [23]:
internals_with_NAPLAN['description'].value_counts()

Common Tasks          11091
Homework               7316
Class Work             3763
Other                  2110
Assessment Task        1768
End of Semester        1664
Quiz                   1376
Assignment              206
Extension               120
Research Project         48
Developmental Task        2
Name: description, dtype: int64

**Number of records with non-NaN numeric results**

In [35]:
na_filt = ~internals_with_NAPLAN['result_numeric'].isna()
internals_with_NAPLAN[na_filt].shape

(13265, 46)

**Number of records with non-NaN results**

In [33]:
na_filt = ~internals_with_NAPLAN['result_description'].isna()
internals_with_NAPLAN[na_filt].shape

(14104, 46)

**Filtering out rows with NaN numeric results**

In [36]:
internal_NAP_numeric = internals_with_NAPLAN[na_filt]
internal_NAP_numeric.shape

(13265, 46)

**Extracting only '_End of Semester_' results**

In [42]:
eos_filt = internal_NAP_numeric['description']=='End of Semester'
internal_NAP_numeric_eos = internal_NAP_numeric[eos_filt]
internal_NAP_numeric_eos.shape

(1646, 46)

In [47]:
internal_NAP_numeric_eos['title'].value_counts().index.tolist()

['Semester 2', 'Semester 1']

In [87]:
from statistics import mean 
stud = '3ff4e152-fe23-90f1-2dc5-795379649da8'
year = 2021
sem = 'Semester 2'
year_sem_filt = (internal_NAP_numeric_eos['school_year']==year) & (internal_NAP_numeric_eos['title']==sem)
stu_filt = (internal_NAP_numeric_eos['stu_uuid']==stud) & (year_sem_filt)
internal_NAP_numeric_eos[stu_filt]['result_numeric'].values[0] 

0.87

In [77]:
internal_NAP_numeric_eos[internal_NAP_numeric_eos['stu_uuid']==stud]

,tnt_uuid,assessment_uuid,ttsess_uuid,stu_uuid,ttclass_uuid,status,name,activate,assessment_result_uuid,result_numeric,...,session_start,session_end,status.1,title,school_year,tnt_uuid.2,ldom_uuid.1,learning_domain,learning_domain_code,learning_domain_enum
0,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e3e986-bc32-0a22-9085-0d6088b6d309,67045c9c-21be-879c-1774-9b6a2d22a73b,3ff4e152-fe23-90f1-2dc5-795379649da8,443eb7a2-e3f6-ca47-669c-e72638cea15a,ASSIGNED,Mathematics Semester Result G08MT4 K,2020-11-26 14:00:00+00:00,c512cee0-6765-6cd8-64d8-63953fcec12c,0.96,...,2020-06-15,2020-12-09,active,Semester 2,2020,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,NaN
1,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e35d50-9b2f-f8ae-6555-58f0949d9b66,67046440-439d-5c20-d62f-8ddc655052cf,3ff4e152-fe23-90f1-2dc5-795379649da8,443d72a3-8eaa-c49e-bdf5-58efb72b9e32,ASSIGNED,Mathematics Semester Result G07MT1 K,2019-06-13 14:00:00+00:00,c508f21c-bdb6-5f80-3b7a-bacfdef687c6,0.97,...,2019-01-29,2019-06-14,active,Semester 1,2019,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,NaN
2,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e44eae-1534-f120-29e1-5d0abff52050,6704c9c1-2c77-0533-e04a-e73fa620c87b,3ff4e152-fe23-90f1-2dc5-795379649da8,443f3130-1ca9-65f0-852c-ab39aeb5be6f,ASSIGNED,Mathematics Semester One Grade G08MT4 K,2020-06-11 14:00:00+00:00,c508f942-d586-1a52-6d89-c228eaf745df,0.92,...,2020-01-31,2020-06-12,active,Semester 1,2020,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,NaN
3,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e49d22-9205-e2b7-afdc-312fd9564810,670498db-d726-8504-5ac1-d28ef68b5bcb,3ff4e152-fe23-90f1-2dc5-795379649da8,443e4335-7376-0acd-2d95-20751949a458,ASSIGNED,Mathematics Semester result G07MT1 K,2019-12-01 14:00:00+00:00,c50b8f96-1903-9cfa-4505-f01ca31d6f43,0.95,...,2019-06-17,2019-12-11,active,Semester 2,2019,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,NaN
4,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e46ea5-e4c8-a3f1-a72d-324c504d8fb3,67044b53-5747-9015-5532-332cc438841a,3ff4e152-fe23-90f1-2dc5-795379649da8,443fa9f5-a241-7aea-31b4-332332ae5592,ASSIGNED,Mathematics A Semester result G09MA4 K,2021-11-24 14:00:00+00:00,c51f0189-2fa0-f43e-48c3-3d5e3c155b60,0.87,...,2021-07-13,2021-12-10,active,Semester 2,2021,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,NaN
5,6d27c189-cc31-4970-a6b2-49019ce66c3f,c4e5d6c1-88b9-681b-8a35-1903b9ead6e8,670487c2-def4-906e-5a16-fb8b205126fb,3ff4e152-fe23-90f1-2dc5-795379649da8,443efd23-c19a-0464-7e7d-6f85f085e2d0,ASSIGNED,Mathematics A Semester Result G09MA4 K,2021-06-16 14:00:00+00:00,c518b632-d7c1-c050-2cc0-f9ffd7d05340,0.83,...,2021-02-01,2021-06-25,active,Semester 1,2021,6d27c189-cc31-4970-a6b2-49019ce66c3f,9409e639-1c34-dbba-0c86-40ceed9af969,Mathematics,MATHS,NaN


**Internal Assessment End-of-Semester Percentiles**

**Populate a new dataframe with each _row as a student_ and each _column as a year_**

In [90]:
stu_uuid = internal_NAP_numeric_eos['stu_uuid'].unique()
school_year = internal_NAP_numeric_eos['school_year'].value_counts().index.tolist()
title = internal_NAP_numeric_eos['title'].value_counts().index.tolist()

internal_clean = {'stu_uuid': []}

for year in school_year:
    internal_clean[year] = []

for stud in stu_uuid:
    internal_clean['stu_uuid'].append(stud)
    for year in school_year:
        percentiles = []
        for sem in title:

            # print(stud, year, sem)
            year_sem_filt = (internal_NAP_numeric_eos['school_year']==year) & (internal_NAP_numeric_eos['title']==sem)
            stu_filt = (internal_NAP_numeric_eos['stu_uuid']==stud) & (year_sem_filt)  

            stu_score = internal_NAP_numeric_eos[stu_filt]['result_numeric'].values
            # print(stu_score)

            if len(stu_score)>0:
                percentiles.append(
                    stats.percentileofscore(internal_NAP_numeric_eos[year_sem_filt]['result_numeric'], stu_score[0], kind='weak')
                ) 
        
        mean_percentile = None
        if len(percentiles)>0:
            mean_percentile = mean(percentiles)
        internal_clean[year].append(mean_percentile)
            

In [92]:
internal_df = pd.DataFrame(internal_clean)
internal_df

,stu_uuid,2021,2020,2019
0,3ff4e152-fe23-90f1-2dc5-795379649da8,78.340974,96.341007,97.545009
1,3ff4feda-755e-f70e-bcd4-f7509d9e851c,6.800538,14.276356,26.562005
2,3ff4c00c-98b9-2878-66a8-b0b0b8a804aa,70.117367,58.142162,74.149503
3,3ff4fd52-852c-a3b0-48ce-1b78aaf3181d,84.598081,90.501591,83.836958
4,3ff4ed76-9d26-3078-57fc-8589e66130c2,21.124152,2.563434,15.740314
...,...,...,...,...
284,3ff4f279-7e51-015f-2610-0976670e4173,30.945046,7.859416,19.533343
285,3ff4285a-ab65-b922-2c9b-dd0b9e53aea8,39.324531,NaN,NaN
286,3ff44d2d-2c90-a073-e7ac-e247f5e9a230,51.028791,75.153071,48.916895
287,3ff40039-4aa0-4514-17c4-02a387932ab7,23.441531,14.258309,10.442172


In [95]:
internal_df.to_csv('internal_df.csv', header=True, index=False)

**NAPAN Numeracy Scores**

**Populate Dataframe with each row as a student and each column as a year**

In [132]:
test_year = externals['test_year'].value_counts().index.tolist()

external_clean = {'stu_uuid': []}

for year in test_year:
    external_clean[year] = []

for stud in stu_uuid:
    external_clean['stu_uuid'].append(stud)
    for year in test_year:
        ext_score = externals[(externals['stu_uuid']==stud) & (externals['test_year']==year)]['percentile'].values
        if len(ext_score)>0:
            external_clean[year].append(ext_score[0]*100)
        else:
            external_clean[year].append(None)

In [133]:
external_df = pd.DataFrame(external_clean)
external_df

,stu_uuid,2021,2019,2017,2015,2012,2013
0,3ff4e152-fe23-90f1-2dc5-795379649da8,91.568297,89.169675,NaN,NaN,NaN,NaN
1,3ff4feda-755e-f70e-bcd4-f7509d9e851c,49.072513,18.050542,18.291055,2.892562,NaN,NaN
2,3ff4c00c-98b9-2878-66a8-b0b0b8a804aa,69.477234,76.173285,NaN,NaN,NaN,NaN
3,3ff4fd52-852c-a3b0-48ce-1b78aaf3181d,98.650927,74.548736,NaN,NaN,NaN,NaN
4,3ff4ed76-9d26-3078-57fc-8589e66130c2,NaN,6.859206,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
284,3ff4f279-7e51-015f-2610-0976670e4173,35.750422,4.693141,NaN,1.377410,NaN,NaN
285,3ff4285a-ab65-b922-2c9b-dd0b9e53aea8,54.300169,NaN,NaN,NaN,NaN,NaN
286,3ff44d2d-2c90-a073-e7ac-e247f5e9a230,63.406408,23.104693,NaN,NaN,NaN,NaN
287,3ff40039-4aa0-4514-17c4-02a387932ab7,19.730185,7.220217,9.612817,NaN,NaN,NaN


In [134]:
externals['test_year'].value_counts()

2021    287
2019    266
2017    137
2015    104
2012      1
2013      1
Name: test_year, dtype: int64

_We have only significant amount of results for 2021 and 2019_

In [135]:
external_df.to_csv('external_df.csv', header=True, index=False)

**Merge Internal and External Results for 2021 in a single dataframe**

In [ ]:
internal_df.set_index('stu_uuid', inplace=True)
external_df.set_index('stu_uuid', inplace=True)

In [142]:
merged2021 = pd.DataFrame({'naplan_2021': external_df[2021]}, index=external_df.index)
merged2021 = pd.concat([internal_df[2021], merged2021], axis=1)

In [146]:
merged2021 = merged2021.dropna()

In [149]:
merged2021

,2021,naplan_2021
stu_uuid,,
3ff4e152-fe23-90f1-2dc5-795379649da8,78.340974,91.568297
3ff4feda-755e-f70e-bcd4-f7509d9e851c,6.800538,49.072513
3ff4c00c-98b9-2878-66a8-b0b0b8a804aa,70.117367,69.477234
3ff4fd52-852c-a3b0-48ce-1b78aaf3181d,84.598081,98.650927
3ff4fc5a-2482-0435-8fb3-10b7db06f63c,51.750107,87.689713
...,...,...
3ff4f279-7e51-015f-2610-0976670e4173,30.945046,35.750422
3ff4285a-ab65-b922-2c9b-dd0b9e53aea8,39.324531,54.300169
3ff44d2d-2c90-a073-e7ac-e247f5e9a230,51.028791,63.406408


In [150]:
merged2021.to_csv('merged_2021.csv', header=True, index=True)

In [152]:
pd.read_csv('merged_2021.csv', index_col='stu_uuid')

,2021,naplan_2021
stu_uuid,,
3ff4e152-fe23-90f1-2dc5-795379649da8,78.340974,91.568297
3ff4feda-755e-f70e-bcd4-f7509d9e851c,6.800538,49.072513
3ff4c00c-98b9-2878-66a8-b0b0b8a804aa,70.117367,69.477234
3ff4fd52-852c-a3b0-48ce-1b78aaf3181d,84.598081,98.650927
3ff4fc5a-2482-0435-8fb3-10b7db06f63c,51.750107,87.689713
...,...,...
3ff4f279-7e51-015f-2610-0976670e4173,30.945046,35.750422
3ff4285a-ab65-b922-2c9b-dd0b9e53aea8,39.324531,54.300169
3ff44d2d-2c90-a073-e7ac-e247f5e9a230,51.028791,63.406408


In [38]:
filt2 = internal_NAP_numeric['stu_uuid'] == internal_NAP_numeric.loc[0, 'stu_uuid']
# internal_NAP_numeric[filt2][~internal_NAP_numeric[filt2]['result_numeric'].isna()]
internal_NAP_numeric[filt2]['description'].value_counts()

Common Tasks       21
End of Semester     6
Assessment Task     4
Quiz                4
Other               2
Homework            2
Name: description, dtype: int64